In [22]:
import numpy as np
import pandas as pd
from os.path import join as oj
import os
import pandas as pd
import sys
import inspect
import datetime
import time
from scipy.stats import percentileofscore
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)
sys.path.append(parentdir + '/modeling')
import load_data
from fit_and_predict import add_preds
from functions import merge_data
from viz import  viz_interactive
import numpy as np
import pandas as pd
from os.path import join as oj
import os
import pandas as pd
import sys
import matplotlib as plt
from scipy.stats import percentileofscore
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
import statsmodels.api as sm

# CHANGE THIS
from exponential_modeling import *
from fit_and_predict import *
#from pmdl_weights import * 

from shared_models import SharedModel


In [2]:
# Load in the Data
# Imputting data for 23rd of May as 13 cases
print('loading data...')
NUM_DAYS_LIST = [1, 2, 3, 4, 5, 6, 7]
df_county = load_data.load_county_level(data_dir=oj(parentdir, 'data'))
#df_county = add_preds(df_county, NUM_DAYS_LIST=NUM_DAYS_LIST,
#                          cached_dir=oj(parentdir, 'data'))

loading data...
loading county-level data...
loaded and merged COVID-19 cases/deaths data successfully


In [3]:
df_uc = pd.read_csv("../data/hospital_level/raw/UC_hospital/UC_Hospitalizations.csv")
# 3/18/20 - 5/28/20, cutting off NaNs
df_uc = df_uc[:72]
df_uc

,Date,UCSF,UCSD,UCLA,UCD,UCI
0,03-18-2020,8.0,10.0,18.0,4.0,1.0
1,03-19-2020,9.0,11.0,20.0,6.0,2.0
2,03-20-2020,12.0,11.0,22.0,7.0,4.0
3,03-21-2020,10.0,10.0,23.0,7.0,5.0
4,03-22-2020,11.0,10.0,21.0,8.0,3.0
...,...,...,...,...,...,...
67,05-24-2020,17.0,38.0,33.0,3.0,25.0
68,05-25-2020,16.0,41.0,33.0,2.0,25.0
69,05-26-2020,16.0,42.0,35.0,2.0,27.0
70,05-27-2020,15.0,40.0,33.0,3.0,29.0


In [4]:
df_zsfg = pd.read_csv("../../covid-19-private-data/ucsf_hospitalization_data.csv", sep="\t")
df_zsfg = df_zsfg[["Date", "ZSFG COVID-19+ (total admitted)"]]
# 3/18/20 - 5/28/20, cutting off beginning days and NaNs
df_zsfg = df_zsfg.iloc[10:82]
df_zsfg

,Date,ZSFG COVID-19+ (total admitted)
10,18-Mar,2
11,19-Mar,1
12,20-Mar,0
13,21-Mar,1
14,22-Mar,2
...,...,...
77,24-May,12
78,25-May,14
79,26-May,15
80,27-May,16


In [5]:
df_uc["ZSFG"] = df_zsfg["ZSFG COVID-19+ (total admitted)"].to_numpy()
df_uc

,Date,UCSF,UCSD,UCLA,UCD,UCI,ZSFG
0,03-18-2020,8.0,10.0,18.0,4.0,1.0,2
1,03-19-2020,9.0,11.0,20.0,6.0,2.0,1
2,03-20-2020,12.0,11.0,22.0,7.0,4.0,0
3,03-21-2020,10.0,10.0,23.0,7.0,5.0,1
4,03-22-2020,11.0,10.0,21.0,8.0,3.0,2
...,...,...,...,...,...,...,...
67,05-24-2020,17.0,38.0,33.0,3.0,25.0,12
68,05-25-2020,16.0,41.0,33.0,2.0,25.0,14
69,05-26-2020,16.0,42.0,35.0,2.0,27.0,15
70,05-27-2020,15.0,40.0,33.0,3.0,29.0,16


In [6]:
train = df_uc[["ZSFG", "UCD", "UCLA", "UCI"]]
test = df_uc[["UCSF", "UCSD"]]

In [15]:
def four_models(df, name, startday, k, three=False, c0=1, mu=0.5):
    """
    df: array-like, hospital data
    name: string, hospital name
    startday: int, index of first prediction, >= 1
    k: int, # of days to predict in the future, >= 1
    three: boolean, default false 
        false - include all four models 
        true - leave CLEP out, CLEP predictions is empty list
    c0: int, constant weighting parameter 
    mu: int, exponential weighting parameter, 0 <= mu <= 1

    Output: 
    list of predictions for linear, separate exp, shared exp, and CLEP models
    """
    lin_future_predictions = []
    sep_exp_future_predictions = []
    shared_exp_future_predictions = []
    ensemble = []
    for i in range(startday,df.shape[0]-k + 1):
        tmp = df[:i] # Should there be an i+1 for predict_future? No
        d = {'Name':[name],'hospitalizations':[tmp]}
        df_mod = pd.DataFrame(data = d)
        # Shared Exponential Predictor
        shared_future_pred = fit_and_predict_shared_exponential(df_mod, 
                                mode = "predict_future",
                                outcome = "hospitalizations",
                                demographic_vars=[],
                                target_day=np.array([k]), verbose=False)
        shared_exp_future_predictions.append(shared_future_pred[0][0])
        # CLEP Predictor
        if three == False:
            ensemble_future_prediction = fit_and_predict_ensemble(df_mod,
                                    target_day = np.array([k]),
                                    outcome = 'hospitalizations', 
                                    methods = [shared_exponential, linear],
                                    mode = 'predict_future', verbose = False,
                                    weight_c0 = c0,
                                    weight_mu = mu)['predicted_hospitalizations_ensemble_'+str(k)].values[0][0]
            ensemble.append(ensemble_future_prediction)
    
        tmp = tmp.reshape(1, tmp.shape[0])
        # Linear Predictor
        lin_pred_future = linear_fit(tmp,'predict_future',
                                target_day=np.array([k]))
        lin_future_predictions.append(lin_pred_future[0][0])
        # Separate Exponential Predictor
        sep_exp_pred_future = exponential_fit(tmp,'predict_future', 
                                target_day=np.array([k]))
        sep_exp_future_predictions.append(sep_exp_pred_future[0][0])

    return [np.array(lin_future_predictions), np.array(sep_exp_future_predictions), 
            np.array(shared_exp_future_predictions), np.array(ensemble)]

def CLEP(df, name, startday, k, c0=1, mu=0.5):
    """
    df: array-like, hospital data
    name: string, hospital name
    startday: int, index of first prediction, >= 1
    k: int, # of days to predict in the future, >= 1
    c0: int, constant weighting parameter 
    mu: int, exponential weighting parameter, 0 <= mu <= 1
    
    Output: 
    predictions for linear, separate exp, shared exp, and CLEP models
    """
    ensemble = []
    for i in range(startday, df.shape[0]-k+1): 
        tmp = df[:i] 
        d = {'Name':[name],'hospitalizations':[tmp]}
        df_mod = pd.DataFrame(data = d)
        ensemble_future_prediction = fit_and_predict_ensemble(df_mod,
                                        target_day = np.array([k]),
                                        outcome = 'hospitalizations', 
                                        methods = [shared_exponential, linear],
                                        mode = 'predict_future', verbose = False,
                                        weight_c0 = c0,
                                        weight_mu = mu)['predicted_hospitalizations_ensemble_'+str(k)].values[0][0]
        ensemble.append(ensemble_future_prediction)
    return ensemble

def errors(preds, true, three=False):
    names = ["linear", "sep exp", "shar exp", "CLEP"]
    for i in range(4):
        if not (three == True and i == 4):
            print(names[i]+" MAE: "+str(mean_absolute_error(preds[i], true)))
            print(names[i]+" log MAE: "+str(mean_absolute_error(np.log(preds[i]+1), np.log(true))))
            print(names[i]+" Med AE: "+str(median_absolute_error(preds[i], true)))
            print(names[i]+" log Med AE: "+str(median_absolute_error(np.log(preds[i]+1), np.log(true))))
    return
    

In [16]:
preds = four_models(df_uc["ZSFG"].to_numpy(), "ZSFG", 22, 5)

In [13]:
CLEP(df_uc["ZSFG"].to_numpy(), "ZSFG", 22, 5)

[37.233926208484505,
 34.40728745633997,
 35.16862514458659,
 33.9282379238261,
 36.240564996396074,
 37.14046085082817,
 28.339396652542177,
 19.298680856148273,
 18.240787619596905,
 26.547182204065415,
 27.18860531118935,
 25.005564222965447,
 25.66104766694133,
 26.739110322385102,
 28.043240016390868,
 25.62753573708482,
 22.952600853798348,
 14.406741949316395,
 14.337520067966212,
 17.23231054917147,
 25.619158447901633,
 27.64150169138934,
 28.913404073168913,
 26.622258983984015,
 21.162010054573148,
 20.85568389468817,
 18.3539538501515,
 19.945204084704784,
 22.8873848916095,
 26.73213877113764,
 21.151782724029147,
 17.17753125867707,
 14.884021564752624,
 16.89749343092433,
 18.660725455356527,
 20.87695645751201,
 21.918851789405423,
 23.960336664949544,
 19.520089188012896,
 16.455971762257597,
 18.317030323227527,
 23.70721108253222,
 16.74500301898788,
 6.782363378598944,
 8.121610972449787,
 13.422069477879877]

In [87]:
errors(preds, df_uc["ZSFG"][22+5-1:].to_numpy())

linear MAE: 7.484782608695652
linear log MAE: 0.4984088161447639
linear Med AE: 6.450000000000008
linear log Med AE: 0.3101841455571708
sep exp MAE: 6.786980792561724
sep exp log MAE: 0.3251537116596896
sep exp Med AE: 6.296423222979066
sep exp log Med AE: 0.3245293662234796
shar exp MAE: 4.9935293179811495
shar exp log MAE: 0.26310543929152413
shar exp Med AE: 4.474585755531457
shar exp log Med AE: 0.24977326163555036
CLEP MAE: 5.48982135075817
CLEP log MAE: 0.281001116173315
CLEP Med AE: 5.602207607418514
CLEP log Med AE: 0.25469204021998726


## ZSFG

In [37]:
days = [3, 5, 7]
startdays = [24, 22, 20]
mus = np.arange(0.2, 1.1, 0.2)
c0s = np.arange(0.25, 1.1, 0.25)

best_mu = mus[0]
best_c0 = c0s[0]
best_error = np.infty

all_errors = []
all_params = []
for i in range(1):
    errors = []
    params = []
    counter = 1
    #want to iterate over pred days too?
    t0 = time.time()
    for c in c0s:
        for m in mus:
            clep_preds = CLEP(df_uc["ZSFG"].to_numpy(), "ZSFG", 24, 3, c0=c, mu=m)
            error = mean_absolute_error(clep_preds, df_uc["ZSFG"][24+3-1:].to_numpy())
            errors.append(error)
            params.append([m, c])
            if error < best_error:
                best_mu = m
                best_c0 = c
                best_error = error
            print("done with "+str(counter)+": "+str([c, m]))
            counter += 1
    all_errors.append(error)
    all_errors.append(params)
t1 = time.time()
print("Time: "+str(t1-t0))
print("best c0: "+str(best_c0)+", best mu: "+str(best_mu)+", best error: "+str(best_error))

done with 1: [0.25, 0.2]
done with 2: [0.25, 0.4]
done with 3: [0.25, 0.6000000000000001]
done with 4: [0.25, 0.8]
done with 5: [0.25, 1.0]
done with 6: [0.5, 0.2]
done with 7: [0.5, 0.4]
done with 8: [0.5, 0.6000000000000001]
done with 9: [0.5, 0.8]
done with 10: [0.5, 1.0]
done with 11: [0.75, 0.2]
done with 12: [0.75, 0.4]
done with 13: [0.75, 0.6000000000000001]
done with 14: [0.75, 0.8]
done with 15: [0.75, 1.0]
done with 16: [1.0, 0.2]
done with 17: [1.0, 0.4]
done with 18: [1.0, 0.6000000000000001]
done with 19: [1.0, 0.8]
done with 20: [1.0, 1.0]
Time: 396.6895475387573
best c0: 1.0, best mu: 0.2, best error: 3.8259776936020287


In [44]:
days = [3, 5, 7]
startdays = [24, 22, 20]
mus = np.arange(0.2, 1.1, 0.2)
c0s = np.arange(0.25, 1.1, 0.25)

all_errors = []
all_params = []
best_mus = []
best_c0s = []
best_errors = []
print("Total counter: ",str(3*len(c0s)*len(mus)))
df=df_uc["ZSFG"].to_numpy()
for i in range(3):
    errors = []
    params = []
    counter = 1
    best_mu = mus[0]
    best_c0 = c0s[0]
    best_error = np.infty
    true_df=df_uc["ZSFG"][startdays[i]+days[i]-1:].to_numpy()
    
    t0 = time.time()
    for c in c0s:
        for m in mus:
            clep_preds = CLEP(df, "ZSFG", startdays[i], days[i], c0=c, mu=m)
            error = mean_absolute_error(clep_preds, true_df)
            errors.append(error)
            params.append([c, m])
            if error < best_error:
                best_mu = m
                best_c0 = c
                best_error = error
            print("done with "+str(counter)+": "+str([c, m]))
            counter += 1
    all_errors.append(error)
    all_errors.append(params)
    best_mus.append(best_mu)
    best_c0s.append(best_c0)
    best_errors.append(best_error)
t1 = time.time()
print("Time: "+str(t1-t0))
print("best c0: "+str(best_c0)+", best mu: "+str(best_mu)+", best error: "+str(best_error))

Total counter:  60
done with 1: [0.25, 0.2]
done with 2: [0.25, 0.4]
done with 3: [0.25, 0.6000000000000001]
done with 4: [0.25, 0.8]
done with 5: [0.25, 1.0]
done with 6: [0.5, 0.2]
done with 7: [0.5, 0.4]
done with 8: [0.5, 0.6000000000000001]
done with 9: [0.5, 0.8]
done with 10: [0.5, 1.0]
done with 11: [0.75, 0.2]
done with 12: [0.75, 0.4]
done with 13: [0.75, 0.6000000000000001]
done with 14: [0.75, 0.8]
done with 15: [0.75, 1.0]
done with 16: [1.0, 0.2]
done with 17: [1.0, 0.4]
done with 18: [1.0, 0.6000000000000001]
done with 19: [1.0, 0.8]
done with 20: [1.0, 1.0]
done with 1: [0.25, 0.2]
done with 2: [0.25, 0.4]
done with 3: [0.25, 0.6000000000000001]
done with 4: [0.25, 0.8]
done with 5: [0.25, 1.0]
done with 6: [0.5, 0.2]
done with 7: [0.5, 0.4]
done with 8: [0.5, 0.6000000000000001]
done with 9: [0.5, 0.8]
done with 10: [0.5, 1.0]
done with 11: [0.75, 0.2]
done with 12: [0.75, 0.4]
done with 13: [0.75, 0.6000000000000001]
done with 14: [0.75, 0.8]
done with 15: [0.75, 1.0]

In [45]:
best_mus

[0.2, 0.2, 0.2]

In [46]:
best_c0s

[1.0, 1.0, 0.75]